## FINGER TAPPING ANALYSIS
##### Data collected from patients with neurodegenerative disorders as well as healthy controls

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
import time
import sys
from tqdm import tqdm
import os
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
import math
from statsmodels.sandbox.stats.multicomp import multipletests
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
import keras
from keras.models import Model
from keras.layers import Input, Conv1D, Flatten, Dropout, MaxPooling1D, Dense
from keras.layers import Activation, BatchNormalization, concatenate
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping
%matplotlib inline

In [3]:
# Read the data

train = scipy.io.loadmat('TrainData.mat')
val = scipy.io.loadmat('ValData.mat')


In [6]:
Xtrain, Ytrain = train['X'], train['Y']
Xval, Yval = val['X'], val['Y']

### Arite, lez make a model

In [115]:
def CNNModel(inputShape, nConvLayers):
    
    input1 = Input(shape = inputShape)

    #convolutions
    x = input1
    for i in range(0,nConvLayers):
        
        nFilters = 256 if i>1 else 64*(i+1)
        inside = 3 if i>1 else 2
        for temp in range(0,inside):
            x = Conv1D(filters = nFilters,
                  kernel_size = 5,
                  padding = 'same',
                  strides = 1,
                  name = 'Conv1x5{}{}'.format(i,temp))(x)
            x = BatchNormalization()(x)
            x = Activation('relu',name='ReLu{}{}'.format(i,temp))(x)
            
        x = MaxPooling1D(2,
                      padding = 'same',
                      strides = 1,
                      name = 'MaxPooling1D{}'.format(i))(x)
        x = Dropout(0.3)(x)

    
    # Fully connected
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Dropout(0.3)(x)
    x = Dense(64)(x)
    x = Dropout(0.3)(x)
    x = Activation('relu', name = 'reLU_dense')(x)
    x = Dense(4)(x)
    x = Activation('softmax',name = 'Softmax')(x)

    m = Model(input1,x)
    return m


In [7]:
def inceptionModel(inputShape):
    
    input1 = Input(shape = inputShape)

    #conv1
    x = Conv1D(filters = 64,
              kernel_size = 5,
              padding = 'same',
              strides = 1,
              name = 'Conv1')(input1)
    x = BatchNormalization()(x)
    x = Activation('relu',name='ReLu1')(x)
    x = Dropout(0.2)(x)

    # inception 2
    x3 = Conv1D(filters = 64,
              kernel_size = 1,
              padding = 'same',
              strides = 1)(x)
    x3 = Conv1D(filters = 64,
               kernel_size = 5,
               padding = 'same',
               strides = 1,
               name = 'Conv1x5')(x3)

    x5 = Conv1D(filters = 64,
              kernel_size = 1,
              padding = 'same',
              strides = 1)(x)
    x5 = Conv1D(filters = 64,
              kernel_size = 7,
              padding = 'same',
              strides = 1,
              name = 'Conv1x7')(x5)

    xmax = MaxPooling1D(3,
              padding = 'same',
              strides = 1,
              name = 'MaxPoolIncept')(x)
    xmax = Conv1D(filters = 64,
              kernel_size = 1,
              padding = 'same',
              strides = 1)(xmax)

    x357 = concatenate([x3,x5,xmax])

    #conv 3
    x = Conv1D(filters = 256,
              kernel_size = 3,
              padding = 'same',
              strides = 1,
              name = 'Conv3')(x357)
    
    x = Activation('relu',name='ReLu3')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.3)(x)
    

    # Fully connected
    x = BatchNormalization()(x)
    x = Flatten()(x)
    x = Dense(128)(x)
    x = Activation('relu', name = 'reLU_dense')(x)
    x = Dropout(0.4)(x)
    x = Dense(4)(x)
    x = Activation('sigmoid',name = 'Softmax')(x)

    m = Model(input1,x)
    return m

In [8]:
def saveModelTopology(model,modelName):
    model_json = model.to_json()
    with open(modelName+'.json', "w") as json_file:
        json_file.write(model_json)
    print("Saved model to {}.json.".format(modelName))
    return


In [9]:
def defCallbacks(weightFile):
    
    checkpoint = ModelCheckpoint(weightFile,
                                 monitor='val_acc',
                                 verbose=1,
                                 save_best_only = True,
                                 save_weights_only = True,
                                 mode='max')
    early = EarlyStopping(monitor='val_acc',
                          patience = 10,
                          verbose = 1,
                          mode='max')
    return [checkpoint, early]
    

In [10]:
def fitModel(model, modelName, Xtrain, Ytrain, Xval, Yval, epochs):
    
    tic = time.time()
    
    # make file name
    tm = time.gmtime()
    weightFile = '{}.{}.{}.{}.{}.{}.h5'.format(modelName,tm[2],tm[1],tm[0],tm[3]+1,tm[4])
    
    #define callbacks
    callbacks = defCallbacks(weightFile)
    
    # FIT THE MODEL
    history = model.fit(x = Xtrain, y = Ytrain,
                        epochs=epochs,
                        batch_size=4,
                        validation_data = (Xval,Yval),
                        callbacks = callbacks)
    toc = time.time()
    print("Finished training in {} min ({} h)".format(round((toc-tic)/60,2),round((toc-tic)/3600,2)))

    
    # Save the weights
    model.save_weights(str(modelName)+'.h5') # ???????
    
    return history

In [120]:
def evaluateModel(model, modelName, Xval, Yval):
    tic = time.time()
    predictions = model.predict(Xval)
    toc = time.time()
    print('Finished prediction in: {} min'.format(round((toc-tic)/60,2)))

    print('Evaluating...')
    score = model.evaluate(Xval,Yval,verbose=1)
    print(score)

    #Save that stuff too pls.
    tm = time.gmtime()    
    predictionFile = 'Predictions-{}.{}.{}.{}.{}.{}.csv'.format(modelName,tm[2],tm[1],tm[0],tm[3]+1,tm[4])

    dfPredicted = pd.DataFrame(predictions)
    dfPredicted = dfPredicted.idxmax(axis =1)
    dfExpected = pd.DataFrame(Yval)
    dfExpected = dfExpected.idxmax(axis =1)
    df = pd.DataFrame({'Predicted':dfPredicted, 'Expected':dfExpected})
    df.to_csv(predictionFile,index = False)
    print('Saved predictions to: ', predictionFile)
    
    bingos = sum(df['Predicted'] ==df['Expected'])
    accRly = 100*bingos/df.shape[0]
    print('Currently your actual accuracy on Xval is: {}%'.format(round(accRly,2)))
    
    return accRly

In [ ]:
histories = []
modelDepths = []
accuracies = []

for nConvLayers in range(3,4):
    model = inceptionModel((Xtrain.shape[1],Xtrain.shape[2]))
    
    #model = CNNModel((Xtrain.shape[1],Xtrain.shape[2]),nConvLayers)
    opt = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True)
    model.compile(optimizer = opt,
             loss='categorical_crossentropy',
             metrics = ['accuracy'])
    model.summary()
    #modelName = 'CNN_{}_Layers_Deep'.format(nConvLayers)
    modelName = 'InceptAttempt'
    saveModelTopology(model,modelName)
    
    print('TRAINING...')
    epochs = 200
    history = fitModel(model,modelName, Xtrain, Ytrain, Xval, Yval, epochs)
    histories.append(history.history)   
    modelDepths.append(nConvLayers)
        
    # check how it went
    print(history.history)

    # plot Accuracy over Epochs
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train Acc','Val Acc'])
    plt.title('Accuracy for {} over epochs'.format(modelName))
    plt.show()

    # plot Loss over Epochs
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train Loss','Val Loss'])
    plt.title('Loss for {} over epochs'.format(modelName))
    plt.show()
    
#     acc = evaluateModel(model, modelName, Xval, Yval)
#     accuracies.append(acc)
    
    
    with open("hist.txt", "w") as f:
        for h in histories:
            f.write(str(h) +"\n")

#     with open("hist.txt", "r") as f:
#         for line in f:
#             histo.append(int(line.strip()))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2000, 6)      0                                            
__________________________________________________________________________________________________
Conv1 (Conv1D)                  (None, 2000, 64)     1984        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 2000, 64)     256         Conv1[0][0]                      
__________________________________________________________________________________________________
ReLu1 (Activation)              (None, 2000, 64)     0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dropout_1 